In [55]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

# COLETA DE DADOS DO SITE DO IBGE

In [56]:
#COLETANDO DADOS DO IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

DadosIBGE = pd.DataFrame(pd.read_html(url, match="Municípios do Rio de Janeiro")[0])

#RENOMEAR COLUNAS DO DF
DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

#ALTERANDO NOME DO INDEX
DadosIBGE.index.name = 'idMunic'

#ALTERANDO INDEX PRA COMEÇAR EM 1
DadosIBGE.index = DadosIBGE.index + 1

#ADICIONAR UM COLUNA DE DATA E HORA DE CARGA
DadosIBGE['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

#DadosIBGE

# CRIANDO O BANCO DE DADOS ODS

In [57]:
#Manipulando o sistema de arquivos
endereco = Path('C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Projetos Sociais\\Projeto Dado Humanizado\\Pascoa com Dados\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

if endereco.exists():
    if (BDODS.exists() and BDDW.exists()):
        print('Bancos de dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')
        

Bancos de dados já existem!


# MANIPULANDO OS BANCOS DE DADOS CRIADOS

In [58]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Criar a tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tbLogMunic',conexaoODS,if_exists="append")

#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [59]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionar somente as colunas para criação da dMunicipio
DadosIBGE = DadosIBGE[['nmMunic','codMunic']]

#Criar a tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()

print('Carga do DW concluída!')

Carga do DW concluída!
